# Análisis de afinidad

El análisis de afinidad se utiliza para determinar qué objetos están juntos con frecuencia. Una aplicación común es el "análisis de canasta" (*basket analysis*) para identificar qué artículos son comprados al mismo tiempo. Otras aplicaciones son detección de fraudes, segmentación de mercados, recomendación de productos, y otras más. 

El algoritmo que utilizaremos será el *algoritmo a priori* que incluye los siguientes pasos:
1. Cálculo de la frecuencia de los ítems. Se identifican los elementos individuales más frecuentes y se descartan aquellos cuyo *soporte* (proporción en que aparece) sea menor a cierto umbral.
2. Generación de conjuntos de ítems frecuentes. Se combinan los ítems frecuentes en pares, tríos, etc., para formar conjuntos más grandes. Se descartan los conjuntos cuyo soporte sea menor al umbral.
3. Extracción de reglas de asociación. Se generan reglas del tipo "si compras X, es probable que compres Y"

## Ejemplo: Recomendación de películas

El archivo `peliculas.csv` contiene información de las evaluaciones de películas realizadas por los usuarios de un sitio web. Cada usuario está representado por un id y no se proporciona ninguna otra información personal. Los datos provienen originalmente de la base "MovieLens Beliefs Dataset 2024" en el sitio http://grouplens.org/datasets/movielens y los datos fueron filtrados para mostrar solo películas del año 2020 y posteriores, con más de 50 evaluaciones. Las calificaciones de cada película va desde 0.5 estrellas hasta 5 estrellas.

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import warnings
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
df = pd.read_excel('https://github.com/adan-rs/amd/raw/main/data/movies2.xlsx')

In [ ]:
# Convertir ratings > 3 en "me gusta"
likes_matrix = df.copy()
likes_matrix['liked'] = (likes_matrix['rating'] > 3).astype(int)
likes_matrix.sample(5)

In [ ]:
# Crear matriz usuario-película
user_movie_matrix = likes_matrix.pivot_table(index='userId',columns='title',
    values='liked', fill_value=0)

# Verificar que todos los valores son binarios (0 o 1)
user_movie_matrix = user_movie_matrix.astype(bool).astype(int)
user_movie_matrix

In [ ]:
# Encontrar conjuntos frecuentes
frequent_itemsets = apriori(user_movie_matrix, 
                          min_support=0.1,  # Ajustar soporte
                          use_colnames=True)

In [ ]:
frequent_itemsets

In [ ]:
# Generar reglas de asociación
rules = association_rules(frequent_itemsets, 
                        metric="confidence",
                        min_threshold=0.5)  # Ajusta este valor según tus necesidades

Support: Frecuencia de aparición conjunta  
Confidence: Probabilidad condicional  
Lift: Mejora sobre la probabilidad aleatoria

In [ ]:
# Ordenar reglas por lift
rules = rules.sort_values('lift', ascending=False)
rules.head(5)

In [ ]:
def recomendar_items(item, rules_df, n_recomendaciones=5):
    """
    Obtiene recomendaciones únicas para un item específico basado en reglas de asociación
    """
    # Filtrar reglas donde el item dado está en los antecedentes
    item_rules = rules_df[rules_df['antecedents'].apply(lambda x: item in str(x))].copy()
    
    if len(item_rules) == 0:
        return "No se encontraron recomendaciones para este item"
    
    # Crear nueva columna con items individuales usando loc
    item_rules.loc[:, 'item'] = item_rules['consequents'].apply(lambda x: list(x)[0])
    
    # Quedarnos con la mejor regla (mayor lift) para cada item
    best_rules = item_rules.sort_values('lift', ascending=False)\
                          .drop_duplicates(subset=['item'], keep='first')
    
    # Seleccionar las columnas relevantes y renombrarlas para mayor claridad
    recommendations = best_rules[['item', 'confidence', 'lift']].copy()
    recommendations = recommendations.head(n_recomendaciones)
    recommendations = recommendations.rename(columns={'item': 'item_recomendado'})
    
    # Formatear los valores numéricos
    recommendations.loc[:, 'confidence'] = recommendations['confidence'].apply(lambda x: f"{x:.2%}")
    recommendations.loc[:, 'lift'] = recommendations['lift'].apply(lambda x: f"{x:.2f}")
    
    return recommendations

In [ ]:
pelicula_ejemplo = 'The Batman (2022)'
recomendaciones = recomendar_items(pelicula_ejemplo, rules)
print(f"Recomendaciones para {pelicula_ejemplo}:")
print(recomendaciones)